
# GoogleNews-vectors-negative300

In [1]:

# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\577342\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:

sentences = [['first', 'sentence'], ['second', 'sentence']]

# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

In [2]:

import time
from gensim.models import word2vec, keyedvectors

# Load Google's pre-trained Word2Vec model

t0 = time.time()

model = keyedvectors.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)

t1 = time.time()
print(t1-t0, time.ctime(t1))

159.8061707019806 Wed Jun 28 13:16:22 2017



I'm going to save this on my hard drive so I don't lose it

In [3]:

t0 = time.time()
model.save('data/GoogleNews-vectors-negative300.saved_gensim_model')
t1 = time.time()
print(t1-t0, time.ctime(t1))

In [4]:

model_from_save = gensim.models.Word2Vec.load('data/GoogleNews-vectors-negative300.saved_gensim_model')

AttributeError: 'KeyedVectors' object has no attribute 'negative'

In [11]:

kidnappers_list = ['kidnapped', 'kidnapping', 'kidnap', 'kidnappers', 'Kidnapped', 'Kidnappers']
model.most_similar(positive=['hostage', 'barricade'], negative=kidnappers_list, topn=10)

[('Nasdaq_NASDAQ_TRIN', 0.22317594289779663),
 ('J.Gordon_##-###', 0.22106090188026428),
 ('Ky.Busch_##-###', 0.2192620038986206),
 ('alcatel_lucent', 0.20970909297466278),
 ('Leo_Melamed_chairman', 0.20759356021881104),
 ('vmware_esx', 0.20723533630371094),
 ('microsoft_frontpage', 0.20516636967658997),
 ('Agrobacterium_mediated_transformation', 0.20399871468544006),
 ('COLLECTORS_UNIVERSE_INC._AND', 0.20379486680030823),
 ('PAPER_DUE', 0.2035014033317566)]

In [12]:

model.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [13]:

model.similarity('woman', 'man')

0.76640122309953518

In [14]:

import pandas as pd
import re

encoding = ['latin1', 'iso8859-1', 'utf-8'][1]
attack_regex = re.compile(r"attack")
gtdb_path = "../AndrewTextClassification/data/csv/GTDB"
csv_path =  gtdb_path + '.csv'
gtdb_df = pd.read_csv(csv_path, encoding=encoding, low_memory=False)
attack_column_list = [column for column in gtdb_df.columns for m in [attack_regex.search(column)] if m]
gtdb_df[attack_column_list]['attacktype1_txt'].unique()

array(['Assassination', 'Hostage Taking (Kidnapping)', 'Bombing/Explosion',
       'Facility/Infrastructure Attack', 'Armed Assault', 'Hijacking',
       'Unknown', 'Unarmed Assault', 'Hostage Taking (Barricade Incident)'], dtype=object)

In [15]:

t0 = time.time()
nonalpha_regex = re.compile(r"[^a-zA-Z]+")
sq_regex = re.compile(r"'")
gtdb_df.fillna(value="", inplace=True)

def concat_independendent_variables(df):
    X = pd.Series([])
    for row_index, row_series in df.iterrows():
        row_concat = row_series.astype('str').str.cat(sep=' ')
        row_concat = sq_regex.sub(r'', row_concat)
        row_concat = nonalpha_regex.sub(r' ', row_concat)
        X = X.append(pd.Series([row_concat]), ignore_index=True)
    
    return X

important_columns = [column for column in gtdb_df.columns if (column not in attack_column_list)]
X = concat_independendent_variables(gtdb_df[important_columns])
super_model = model.train(X.tolist())

t1 = time.time()
print(t1-t0, time.ctime(t1))

AttributeError: 'KeyedVectors' object has no attribute 'train'